<h2>Checking the arithmetic standard conjectures for Grassmannians</h2>

**Authors**: Paolo Dolce, Roberto Gualdi and Riccardo Pengo

**Summary**: This code checks whether the arithmetic Chow rings $\widehat{\mathrm{CH}}^\ast(G(m,n))$ of the Grassmannian $G(m,n)$ defined over $\mathbb{Z}$ satisfy the arithmetic standard conjectures of Gillet and Soulé. To do so, we use an explicit additive basis for the Chow ring $R_{m,n} = \mathrm{CH}(\overline{G(m,n)})$ of the canonical Arakelov variety $\overline{G(m,n)}$ associated to $G(m,n)$, which corresponds to the presentation appearing in the article "*Explicit Arakelov-Chow rings of Grassmannians, and arithmetic standard conjectures on  projective space*".

---------------------------
First of all, we import two functions we need:
* `groupby`, from the Python module `itertools`, which can group the elements of a list according to the value of a function;
* `kostka_number_symmeytrica`, from the `symmetrica` library in SAGE, which computes the [Kostka number](https://en.wikipedia.org/wiki/Kostka_number) $K_{\lambda,\mu}$ relative to two partitions $\lambda, \mu$, which counts the number of [semi-standard Young tableaux](https://en.wikipedia.org/wiki/Young_tableau#Tableaux) of shape $\lambda$ and weight $\mu$.

In [1]:
from itertools import groupby
from sage.libs.symmetrica.symmetrica import kostka_number_symmetrica as kostka

Recall now that $R_{m,n}$ is generated as an algebra by variables $\hat{x}_1,\dots,\hat{x}_n,\varepsilon$, such that $x_i$ has degree $i$, and $\varepsilon$ has degree one. We denote monomials in these variables as $\widehat{\mathbf{x}}^v := \hat{x}_1^{v_1} \cdots \hat{x}_n^{v_n}$ where $v = (v_1,\dots,v_n)$.

Then, we define two auxiliary functions:
* `weight(v)` which given a vector $(v_1,\dots,v_n)$ returns $\sum_{i=1}^n i \cdot v_i$. This is precisely the weight of the monomial $\widehat{\mathbf{x}}^v$ in our ring.
* `harm_list(n)` which returns the list $[H_0,\dots,H_n]$ where $H_0 = 0$ and $H_k := \sum_{j=1}^k 1/j$ is the $k$-th harmonic number.

In [2]:
def weight(v):
    return(sum([v[k-1]*k for k in (1..len(v))]))

def harm_list(n):
    H = [0]
    for k in (1..n):
        H.append(H[-1]+1/k)
    return H

Now, the function `B(m,n)` computes the list $\mathcal{B}_{m,n}$ of all those vectors $v \in \mathbb{N}^n$ such that $\sum_{i=1}^n v_i \leq m$. It is known (by the work of Borel) that the monomials $\{\mathbf{x}^v \colon v \in \mathcal{B}_{m,n}\}$ form an additive basis of the geometric Chow ring $\mathrm{CH}^\ast(G(m,n))$. Then, we sub-divide the list $\mathcal{B}_{m,n}$ into a list of lists, whose $p$-th component $\mathcal{B}_{m,n}^p$ consists of the vectors $v \in \mathcal{B}_{m,n}$ having weight $p$.

In [3]:
def B(m,n):
    if (m<0) or (n<0):
        return []
    l = [vector(v) for v in sum([IntegerVectors(j,n).list() for j in (0..m)],[])]
    l.sort(key=weight)
    return [list(x) for _,x in groupby(l,key=weight)]

As a preliminary exercise, let us see how we can check algorithmically the validity of the standard conjectures on the geometric Chow ring $R_{m,n} = \mathrm{CH}^\ast(G(m,n))$.
To do so, we define first of all a function `Vgeom(m,n)` which returns two lists: 
* $Z_{m,n}$, which is the union, over $p \in \{0,\dots,\lfloor n m/2 \rfloor\}$, of the images of the maps $\mathfrak{h}_p \colon \mathcal{B}_{m,n}^p \times \mathcal{B}_{m,n}^p \to \mathbb{N}^n$ defined by $(v,w) \mapsto v + w + (n m - 2 p) e_1$ where $e_1 = (1,0,\dots,0)$ denotes the vector corresponding to the monomial $x_1$. Note that all these vectors have maximal weight $n m$. Moreover, note that the maps $\mathfrak{h}_p$ correspond to the maps $R_{m,n}^p \otimes R_{m,n}^p \to R_{m,n}^{m n}$ given by $(\alpha,\beta) \mapsto \alpha \cdot \beta \cdot x_1^{m n - p}$.
* $M_{m,n}$ is a list of square matrices $M_{m,n}^p$ of size $\dim(R_{m,n}^p)$, whose $(i,j)$-th entry is the index of the vector $\mathcal{B}_{m,n}^p[i] + \mathcal{B}_{m,n}^p[j] + (n m - 2 p) e_1$ inside the list $Z_{m,n}$.

In [4]:
def Vgeom(m,n):
    if (m<0) or (n<0):
        return []
    L = B(m,n)
    e = vector([1]+[0]*(n-1))
    Z = []
    M = [matrix(len(x)) for x in L[0:floor(n*m/2)+1]]
    for p in (0..floor(n*m/2)):
        l = L[p]
        for i in range(len(l)):
            for j in range(i,len(l)):
                z = l[i]+l[j]+(n*m-2*p)*e
                if z in Z:
                    M[p][i,j] = Z.index(z)
                    M[p][j,i] = M[p][i,j]
                else:
                    Z.append(z)
                    M[p][i,j] = len(Z)-1
                    M[p][j,i] = M[p][i,j]
    return Z,M

Now, in order to compute the Hodge-Riemann bilinear form on $R_{m,n}^p$, we need to compute the geometric degree of a vector $v \in R_{m,n}^{m n}$.

Using for instance the geometric Pieri rule, it is easy to see that $\deg(\mathbf{x}^v) = K_{(n)^m,\mu(v)}$ is a [Kostka number](https://en.wikipedia.org/wiki/Kostka_number), where $(n)^m = (n,\dots,n)$ is the rectangular partition of $m n$ having $m$ rows and $n$ columns, whereas $\mu(v) = (n,\dots,n,n-1,\dots,n-1,\dots,1,\dots,1)$ is the partition given by $v_n$ copies of $n$, followed by $v_{n-1}$ copies of $n-1$, and so on.

Note that the identity $\deg(\mathbf{x}^v) = K_{(n)^m,\mu(v)}$ can also be derived using the fact that $\mathbf{x}^v = e_{\mu(v)}(x_1,\dots,x_n)$, where $e_{\mu}$ denotes the elementary symmetric function associated to a partition $\mu$. More precisely, we have the general identity of symmetric functions $e_\mu = \sum_{\lvert \lambda \rvert = \lvert \mu \rvert} K_{\lambda^\ast,\mu} s_\lambda$, where $s_\lambda$ denotes the [Schur symmetric function](https://en.wikipedia.org/wiki/Schur_polynomial) and $\lambda^\ast$ denotes the conjugate of $\lambda$. Now, notice that if $\mu = \mu(v)$, the unique partition $\lambda$ such that $\lambda^\ast \geq \mu(v)$ is given by $\lambda = (m)^n$. Since $K_{\alpha,\beta} = 0$ unless $\alpha \geq \beta$, this shows that $\mathbf{x}^v = K_{(n)^m,\mu(v)} x_n^m$. Since $\deg(x_n^m) = 1$, this allows us to conclude.

We implemented this formula in the function `gdeg(v)`, which returns $\deg(\mathbf{x}^v)$, computed as a Kostka number as explained above.

In [5]:
def gdeg(r):
    n = len(r)
    p = weight(r)
    if p%n != 0:
        raise ValueError("The weight must be a multiple of the length")
    m = p//n
    l = vector([n for j in (1..m)])
    w = sum([[j]*r[j-1] for j in (1..n)],[])
    return kostka(l,w)

Now, using the geometric degree function described above, we can compute the matrices representing the Hodge-Riemann forms $R_{m,n}^p \otimes R_{m,n}^p \to \mathbb{Z}$, which are defined by $(\alpha,\beta) \mapsto \deg(\alpha \cdot \beta \cdot x_1^{n m - 2 p})$. Indeed, to do so, it suffices to replace each entry of the matrix $M_{m,n}^p$, which corresponds to a vector in $Z_{m,n}$, with the degree of this vector. This is implemented in the following function `aHodge(m,n)`.

In [6]:
def gHodge(m,n):
    Z,M = Vgeom(m,n)
    D = [gdeg(v) for v in Z]
    for p in range(len(M)):
        d = M[p].dimensions()[0]
        for i in range(d):
            for j in range(d):
                M[p][i,j] = D[M[p][i,j]]
    return M

Now that we have the list of matrices representing the Hodge-Riemann bilinear forms $Q_{m,n}^p$, we can: 
* show that each $Q_{m,n}^p$ is not degenerate, *i.e.* that these matrices are non-singular, for every $p$;
* compute the signatures $s_{m,n}^p$ of $Q_{m,n}^p$, defined as the difference between the numer of positive and negative eigenvalues, and show that $s_{m,n}^p = (-1)^p \left( 1 + \sum_{q = 1}^{p} (-1)^{q} (\dim(R_{m,n}^q) - \dim(R_{m,n}^{q-1})) \right)$, as shown by Adiprasito, Huh and Katz.

If we do so, then we will have proven (in an algorithmic way) the standard conjectures for the ring $R_{m,n}$.

-------

In order to do so, the first step is to compute the Hilbert function $\sum_{p = 0}^{m n} \dim(R_{m,n}^p) t^p = \prod_{j = 1}^n \frac{1 - t^{m+j}}{1-t^j}$. This is implemented in the function `HilbGeom(m,n)`.

In [7]:
def HilbGeom(m,n):
    R.<t> = PolynomialRing(ZZ)
    return R(product([1-t^(m+j) for j in (1..n)])/product([1-t^j for j in (1..n)])).coefficients()

Then, we will define a function `Signature(v)`, which takes as input a vector $v = (v_0,\dots,v_d)$ and outputs the vector $s$ given by $s_p = (-1)^p (1+\sum_{q = 1}^p (-1)^q (v_q - v_{q-1}))$, truncated at half of the lenth of $v$.

In [8]:
def Signature(v):
    d = len(v)-1
    w = [v[i]-v[i-1] for i in (1..floor(d/2))]
    w = [1]+w
    s = [((-1)^q) * sum([(-1)^p*w[p] for p in (0..q)]) for q in (0..floor(d/2))]
    return s

Finally, we are able to algorithmically check the geometric standard conjectures for $G(m,n)$.

In [9]:
def Check_GST(m,n):
    s1 = Signature(HilbGeommm(m,n))
    s2 = []
    H = gHodge(m,n)
    for h in H:
        if h.is_singular():
            return H,False
        s2.append(QuadraticForm(h).signature())
    if s1 == s2:
        return H,True
    else:
        return H,False

-------
Now, let's turn to the problem of checking the validity of the arithmetic standard conjectures algorithmically. 

First of all, we need an arithmetic analogue of the function `Vgeom(m,n)`, which should provide us with a suitable list of monomials. To write it down, observe that a basis of $\widehat{R}_{m,n}^p$ is given by the monomials: 
$$\{\widehat{\mathbf{x}}^v,\varepsilon \widehat{\mathbf{x}}^w \colon v \in \mathcal{B}_{m,n}^p, \ w \in \mathcal{B}_{m,n}^{p-1} \}$$
and thus it makes sense to consider these monomials separately.

For this reason, the function `Varith(m,n)` outputs two lists $\hat{Z}_{m,n}$ and $\hat{M}_{m,n}$ of length two, such that:
* $\hat{Z}_{m,n}[0]$ is a list of vectors of weight $n m + 1$, corresponding to the images of the maps $\mathcal{B}_{n,m}^p \times \mathcal{B}_{m,n}^p \to \mathcal{B}_{m,n}^{m n + 1}$ given by $(v,w) \mapsto v + w + (m n + 1 - 2 p) e_1$
* $\hat{Z}_{m,n}[1]$ is a list of vectors of weight $n m$, corresponding to the images of the maps $\mathcal{B}_{n,m}^p \times \mathcal{B}_{m,n}^{p-1} \to \mathcal{B}_{m,n}^{m n}$ given by $(v,w) \mapsto v + w + (m n + 1 - 2 p) e_1$
* $\hat{M}_{m,n}[0]$ is a list of square matrices $\hat{A}_{m,n}^p$ of size $\dim(R_{m,n}^p)$, whose $(i,j)$-th entry is the index of the vector $\mathcal{B}_{m,n}^p[i] + \mathcal{B}_{m,n}^p[j] + (n m + 1 - 2 p) e_1$ inside the list $\hat{Z}_{m,n}[0]$.
* $\hat{M}_{m,n}[1]$ is a list of matrices $\hat{B}_{m,n}^p$ of size $\dim(R_{m,n}^p) \times \dim(R_{m,n}^{p-1})$, whose $(i,j)$-th entry is the index of the vector $\mathcal{B}_{m,n}^p[i] + \mathcal{B}_{m,n}^{p-1}[j] + (n m + 1 - 2 p) e_1$ inside the list $\hat{Z}_{m,n}[1]$.

In [10]:
def Varith(m,n):
    if (m<0) or (n<0):
        return []
    L = B(m,n)
    e = vector([1]+[0]*(n-1))
    Z = [[(n*m+1)*e],[]]
    M = [[matrix(QQ,1,1,[0]),matrix(QQ,0)]]+[[matrix(len(L[p])),matrix(len(L[p]),len(L[p-1]))] for p in (1..floor((n*m+1)/2))]
    for p in (1..floor((n*m+1)/2)):
        l = L[p]
        for i in range(len(l)):
            for j in range(i,len(l)):
                z = l[i]+l[j]+(n*m+1-2*p)*e
                if z in Z[0]:
                    M[p][0][i,j] = Z[0].index(z)
                    M[p][0][j,i] = M[p][0][i,j]
                else:
                    Z[0].append(z)
                    M[p][0][i,j] = len(Z[0])-1
                    M[p][0][j,i] = M[p][0][i,j]
        l2 = L[p-1]
        for i in range(len(l)):
            for j in range(len(l2)):
                z = l[i]+l2[j]+(n*m+1-2*p)*e
                if z in Z[1]:
                    M[p][1][i,j] = Z[1].index(z)
                else:
                    Z[1].append(z)
                    M[p][1][i,j] = len(Z[1])-1
    return Z,M

Now, as we did in the geometric case, we must be able to compute the arithmetic degree $\widehat{\deg}(\widehat{\mathbf{x}}^v)$ for every $v \in \mathcal{B}_{m,n}^{m n + 1}$.

In order to do so, we will use the following formula, due to Tamvakis:
$$2 \cdot \widehat{\deg}(\widehat{\mathbf{x}}^v) = K_{((n)^m,1),\mu(v)} \sum_{k=n}^{m+n-1} H_k + \sum_{(a,b,j) \in T_{m,n}} K_{\lambda_{a,b,j}^\ast,\mu(v)} (-1)^{n+b+j} (H_{m+n-a-j-1}-H_{m+n-a-b-1})$$
where $T_{m,n}$ denotes the set of vectors $(a,b,j) \in \mathbb{N}^3$ such that $a < m, j < b < n$ and $a + b + j < m+n$. We compute this set in the following function `Tam(m,n)`.

In [11]:
def Tam(m,n):
    L = list()
    for a in (0..m-1):
        for j in (0..n-1):
            for b in (j+1..min(n-1,m+n-a-j-1)):
                L.append(vector((a,b,j)))
    return L

Now, let us implement Tamvakis's formula:
$$2 \cdot \widehat{\deg}(\widehat{\mathbf{x}}^v) = K_{((n)^m,1),\mu(v)} \sum_{k=n}^{m+n-1} H_k + \sum_{(a,b,j) \in T_{m,n}} K_{\lambda_{a,b,j}^\ast,\mu(v)} (-1)^{n+b+j} (H_{m+n-a-j-1}-H_{m+n-a-b-1})$$
in the function `adeg(v)`, which has optional parameters `T=None` and `H=None`. If they are not passed, they will be computed to be $T = T_{m,n}$ and $H = [H_0,\dots,H_{m+n-1}]$, where $H_0 := 0$ and $H_k := \sum_{j=1}^k 1/j$ is the $k$-th harmonic number. Moreover, as before, we have that $\mu(v)$ is the parition containing $v_i$-times the number $i$. Finally, for every $(a,b,j) \in T_{m,n}$ we let $\lambda_{a,b,j} := ((m-1)^{n-1},a)+(1)^b+(m+n-a-b-j,1^j)$, where the sum of two partitions is defined component-wise.

In [12]:
def adeg(r,T=None,H=None):
    n = len(r)
    p = weight(r)
    if (p-1)%n != 0:
        raise ValueError("The weight must be congruent to one modulo the length")
    m = (p-1)//n
    
    # Initialize required variables
    if T is None:
        T = Tam(m,n)
    if H is None:
        H = harm_list(m+n-1)
    w = sum([[j]*r[j-1] for j in (1..n)],[])
    s = 0
    
    # Compute first summand
    l = [n]*m+[1]
    s += kostka(l,w) * sum([H[k] for k in (n..m+n-1)])
    
    # Compute other summands
    for (a,b,j) in T:
        l = [sum(x) for x in zip([m-1]*(n-1)+[a],[1]*b+[0]*(n-b),[m+n-a-b-j]+[1]*j+[0]*(n-j-1))]
        l = Partition(l).conjugate()
        s += (-1)^(n+b+j) * kostka(l,w) * (H[m+n-a-j-1] - H[m+n-a-b-1])
    return s/2

Now, as before, we want to compute the matrices $\hat{H}_{m,n}^p$ associated to the Hodge-Riemann forms $\hat{R}_{m,n}^p \otimes \hat{R}_{m,n}^p \to \mathbb{Q}$ defined by $(\alpha,\beta) \mapsto \widehat{\deg}(\alpha \cdot \beta \cdot \hat{x}_1^{m n + 1 - 2 p})$, with respect to the basis $\{\widehat{\mathbf{x}}^v,\varepsilon \widehat{\mathbf{x}}^w \colon v \in \mathcal{B}_{m,n}^p, \ w \in \mathcal{B}_{m,n}^{p-1} \}$. Since this basis is naturally split in two, it is also natural to view the matrices $\hat{H}_{m,n}^p$ as block matrices, such that:
* the $(i,j)$-th entry of the upper-left block corresponds to $\widehat{\deg}(\widehat{\mathbf{x}}^{v_i+v_j+(n m + 1 - 2 p) e_1})$ where $v_i = \mathcal{B}_{m,n}^p[i]$ and $v_j = \mathcal{B}_{m,n}^p[j]$. Hence, we can compute this matrix simply by substituting each entry of the matrix $\hat{A}_{m,n}^p$ by the arithmetic degree of the corresponding vector in $\widehat{Z}_{m,n}[0]$
* the $(i,j)$-th entry of the upper-right block corresponds to $\widehat{\deg}(\varepsilon\widehat{\mathbf{x}}^{v_i+w_j+(n m + 1 - 2 p) e_1})$ where $v_i = \mathcal{B}_{m,n}^p[i]$ and $w_j = \mathcal{B}_{m,n}^{p-1}[j]$. Moreover, we know that $\widehat{\deg}(\varepsilon \widehat{\mathbf{x}}^r) = \deg(x^r)/2$ for every $r \in \mathbb{N}^n$ of weight $m n$. Hence, we can compute this matrix simply by substituting each entry of the matrix $\hat{B}_{m,n}^p$ by the geometric degree of the corresponding vector in $\widehat{Z}_{m,n}[1]$
* the matrix in the lower-left block is the transpose of the matrix in the upper-right block, because the Hodge-Riemann form is symmetric
* the $(i,j)$-th entry of the matrix in the lower-right block corresponds to $\widehat{\deg}((\varepsilon \widehat{\mathbf{x}}^{w_i}) \cdot (\varepsilon \widehat{\mathbf{x}}^{w_j}) \cdot \widehat{\mathbf{x}}^{(m n + 1 - 2 p) e_1})$. Hence, the lower-right block is identically zero, because $\varepsilon^2 = 0$.

All of this is implemented in the following function `aHodge(m,n)`.

In [13]:
def aHodge(m,n):
    Z,M = Varith(m,n)
    T = Tam(m,n)
    H = harm_list(m+n-1)
    Darith = [adeg(v,T,H) for v in Z[0]]
    Dgeom = [gdeg(v) for v in Z[1]]
    M[0] = matrix(QQ,[Darith[M[0][0][0,0]]])
    for p in (1..len(M)-1):
        d = M[p][0].dimensions()[0]
        M[p][0] = matrix(QQ,M[p][0])
        for i in range(d):
            for j in range(d):
                M[p][0][i,j] = Darith[M[p][0][i,j]]
        d1,d2 = M[p][1].dimensions()
        M[p][1] = matrix(QQ,M[p][1])
        for i in range(d1):
            for j in range(d2):
                M[p][1][i,j] = Dgeom[M[p][1][i,j]]/2
        M[p] = (-1)^p*block_matrix([[M[p][0],M[p][1]],[M[p][1].transpose(),0]])
    return M

Now, as before, we just need to check that the Hodge-Riemann matrices $\hat{H}_{m,n}^p$ are non-singular, and have the correct signature. To do so, we need to find the Hilbert function of $\hat{R}_{m,n}$. Since $\dim(\hat{R}_{m,n}^p) = \dim(R_{m,n}^p) + \dim(R_{m,n}^{p-1})$, we simply have that $\sum_{p=0}^{m n + 1} \dim(\widehat{R}_{m,n}^p) t^p = (1+t) \prod_{j=1}^n \frac{1-t^{m+j}}{1-t^j}$. This is implemented in the following function `HilbArithm(m,n)`.

In [14]:
def HilbArithm(m,n):
    R.<t> = PolynomialRing(ZZ)
    P = R(product([1-t^(m+j) for j in (1..n)])/product([1-t^j for j in (1..n)]))
    P = (1+t)*P
    return P.coefficients()

Finally, we simply implement a function `Check_AST(m,n)`, which checks that the Hodge-Riemann matrices are non-degenerate and have the correct signature.

In [15]:
def Check_AST(m,n):
    s1 = Signature(HilbArithm(m,n))
    s2 = []
    H = aHodge(m,n)
    for h in H:
        if h.is_singular():
            return H,False
        s2.append(QuadraticForm(h).signature())
    if s1 == s2:
        return H,True
    else:
        return H,False

The aim of the following function is to compute more quickly Kostka numbers, using the recursive formula of:
* Petrović, Z. Z., Prvulović, B. I., & Radovanović, M. (2015). Multiplication in the cohomology of Grassmannians via Gröbner bases. Journal of Algebra, 438, 60‑84. https://doi.org/10.1016/j.jalgebra.2015.04.031

In [28]:
def aKostka(m,n):
    T = Tam(m,n)
    T.sort(key=sum,reverse=True)
    L = S(m*n+1,n)
    L.sort(key=sum)
    M = []
    return M